Import modules

In [1]:
# public
import os
import argparse
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable as V

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# custom
from model_layer import Vgg16_all_layer, Vgg19_all_layer, Res152_all_layer, Dense169_all_layer
from generator import GeneratorResnet
from dct import *
# from utils import *
from loader_checkpoint import *

# logging
import logging, json
with open("logging_config.json", "rt") as file:
    config = json.load(file)
logging.config.dictConfig(config)
logger = logging.getLogger()

ArgumentParser

In [7]:
## Loaded pretrained generator options
parser0 = argparse.ArgumentParser(description='Transferable Perturbation via Frequency Manipulation')
parser0.add_argument('--epochs', type=int, default=0, help='Model checkpoint epoch number')
parser0.add_argument('--eps', type=int, default=10, help='Perturbation budget (0~255)')
parser0.add_argument('--model_type', type=str, default='vgg16', help='Victim model: vgg16, vgg19, res152, dense169')
parser0.add_argument('--RN', type=lambda x: (str(x).lower() == 'true'), default=False, help='If true, activating the Random Normalization module in training phase')
parser0.add_argument('--DA', type=lambda x: (str(x).lower() == 'true'), default=False, help='If true, activating the Domain-agnostic Attention module in training phase')
parser0.add_argument('--FA', type=lambda x: (str(x).lower() == 'true'), default=True, help='If true, activating the Frequency Augmentation module in training phase')
parser0.add_argument('--iter_ckpt', type=lambda x: (str(x).lower() == 'true'), default=False, help='If true, Model checkpoint with iteration number')
parser0.add_argument('--iter', type=int, default=10000, help='Save Model checkpoint iteration number')
args0 = parser0.parse_args(args=[])
logger.info(args0)

2023-01-11 13:30:37,766 - root - INFO - Namespace(DA=False, FA=True, RN=False, epochs=0, eps=10, iter=10000, iter_ckpt=False, model_type='vgg16')


In [8]:
## New generator training options
parser = argparse.ArgumentParser(description='Transferable Perturbation via Frequency Manipulation')
parser.add_argument('--train_dir', default='../dataset/imagenet/train', help='Path for imagenet training data')
parser.add_argument('--batch_size', type=int, default=16, help='Batch size')
parser.add_argument('--epochs', type=int, default=1, help='Number of training epochs')
parser.add_argument('--lr', type=float, default=0.0002, help='Initial learning rate') # default=0.0002
parser.add_argument('--eps', type=int, default=10, help='Perturbation budget (0~255)')
parser.add_argument('--model_type', type=str, default='vgg16', help='Victim model: vgg16, vgg19, res152, dense169')
parser.add_argument('--RN', type=lambda x: (str(x).lower() == 'true'), default=False, help='If true, activating the Random Normalization module in training phase')
parser.add_argument('--DA', type=lambda x: (str(x).lower() == 'true'), default=False, help='If true, activating the Domain-agnostic Attention module in training phase')
parser.add_argument('--FA', type=lambda x: (str(x).lower() == 'true'), default=True, help='If true, activating the Frequency Augmentation module in training phase')
parser.add_argument("--rho", type=float, default=0.5, help="Tuning factor")
parser.add_argument("--sigma", type=float, default=16.0, help="Std of random noise")
parser.add_argument('--iter_ckpt', type=lambda x: (str(x).lower() == 'true'), default=False, help='If true, Model checkpoint with iteration number')
parser.add_argument('--iter', type=int, default=10000, help='Save Model checkpoint iteration number')
args = parser.parse_args(args=[])
logger.info(args)

2023-01-11 13:30:41,468 - root - INFO - Namespace(DA=False, FA=True, RN=False, batch_size=16, epochs=1, eps=10, iter=10000, iter_ckpt=False, lr=0.0002, model_type='vgg16', rho=0.5, sigma=16.0, train_dir='../dataset/imagenet/train')


Control the seed

In [5]:
# def setup_seed(seed):
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
#     torch.backends.cudnn.deterministic = True

# setup_seed(0)

Define the victim classification model

In [ ]:
if args.model_type == 'vgg16':
    model = Vgg16_all_layer.Vgg16()
    layer_idx = 16 # Maxpooling.3
elif args.model_type == 'vgg19':
    model = Vgg19_all_layer.Vgg19()
    layer_idx = 18 # Maxpooling.3
elif args.model_type == 'res152':
    model = Res152_all_layer.Resnet152()
    layer_idx = 5 # Conv3_8
elif args.model_type == 'dense169':
    model = Dense169_all_layer.Dense169()
    layer_idx = 6 # Denseblock.2
else:
    raise Exception('Check the model_type')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

Define the generative attack model/optimizer/dataset

In [ ]:
### Model, Optimizer

### From scratch
# netG = GeneratorResnet()
# netG = nn.DataParallel(netG, device_ids=[0,1,2,3]) # multi-GPU
# netG = netG.to(device)

### Load the pretrained generator
netG = load_gan(args0, 'imagenet')
netG = netG.to(device)

optimG = optim.Adam(netG.parameters(), lr=args.lr, betas=(0.5, 0.999))


if args.RN and args.DA:
    save_checkpoint_suffix = 'BIA+RN+DA'
elif args.RN:
    save_checkpoint_suffix = 'BIA+RN'
elif args.DA:
    save_checkpoint_suffix = 'BIA+DA'
elif args.FA:
    save_checkpoint_suffix = 'BIA+FA'
else:
    save_checkpoint_suffix = 'BIA'

# Data, Transform
scale_size = 256
img_size = 224
data_transform = transforms.Compose([
    transforms.Resize(scale_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
])

train_dir = args.train_dir
train_set = datasets.ImageFolder(train_dir, data_transform)
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_size = len(train_set)
logger.info('Training data size:', train_size)

def default_normalize(t):
    t[:, 0, :, :] = (t[:, 0, :, :] - 0.485) / 0.229
    t[:, 1, :, :] = (t[:, 1, :, :] - 0.456) / 0.224
    t[:, 2, :, :] = (t[:, 2, :, :] - 0.406) / 0.225
    return t

def normalize(t, mean, std):
    t[:, 0, :, :] = (t[:, 0, :, :] - mean) / std
    t[:, 1, :, :] = (t[:, 1, :, :] - mean) / std
    t[:, 2, :, :] = (t[:, 2, :, :] - mean) / std
    return t

Train

In [ ]:
save_checkpoint_dir = 'saved_models/{}'.format(args.model_type)
if not os.path.exists(save_checkpoint_dir):
    os.makedirs(save_checkpoint_dir)

for epoch in range(args.epochs):
    running_loss = 0
    for i, (img, _) in enumerate(train_loader):
        img = img.to(device)
        netG.train()
        optimG.zero_grad()
        
        if args.FA and i%2==1:
            gauss = (torch.randn(img.size()[0], 3, img_size, img_size) * (args.sigma / 255)).to(device)
            mask = (torch.rand_like(img) * 2 * args.rho + 1 - args.rho).to(device)
            
            img_dct = dct_2d(img + gauss).to(device)
            img_idct = idct_2d(img_dct * mask)
            img_idct = V(img_idct, requires_grad=True)
            img = img_idct
            
        else:
            pass
        
        # adversarial translation        
        adv = netG(img)
        adv = torch.min(torch.max(adv, img - args.eps/255.0), img + args.eps/255.0)
        adv = torch.clamp(adv, 0.0, 1.0)
        

        # if args.FA:
        #     gauss = (torch.randn(img.size()[0], 3, img_size, img_size) * (args.sigma / 255)).to(device)
        #     mask = (torch.rand_like(img) * 2 * args.rho + 1 - args.rho).to(device)
            
        #     img_dct = dct_2d(img + gauss).to(device)
        #     img_idct = idct_2d(img_dct * mask)
        #     img_idct = V(img_idct, requires_grad=True)
        #     img = img_idct
            
        #     adv_dct = dct_2d(adv + gauss).to(device)
        #     adv_idct = idct_2d(adv_dct * mask)
        #     adv_idct = V(adv_idct, requires_grad=True)
        #     adv = adv_idct
        # else:
        #     pass
        
        if args.RN:
            mean = np.random.normal(0.50, 0.08) # default=(0.50, 0.08) 
            std = np.random.normal(0.75, 0.08) # default=(0.75, 0.08)
            adv_out_slice = model(normalize(adv.clone(), mean, std))[layer_idx]
            img_out_slice = model(normalize(img.clone(), mean, std))[layer_idx]
        else:
            adv_out_slice = model(default_normalize(adv.clone()))[layer_idx]
            img_out_slice = model(default_normalize(img.clone()))[layer_idx]
        
        if args.DA:
            attention = abs(torch.mean(img_out_slice, dim=1, keepdim=True)).detach()
        else:
            attention = torch.ones(adv_out_slice.shape).cuda()
            
        loss = torch.cosine_similarity((adv_out_slice*attention).reshape(adv_out_slice.shape[0], -1), 
                                       (img_out_slice*attention).reshape(img_out_slice.shape[0], -1)).mean()
        loss.backward()
        optimG.step()
        
        # Every 100 iterations
        if i % 100 == 0:
            logger.info('Epoch: {0} \t Batch: {1} \t loss: {2:.5f}'.format(epoch, i, running_loss/100))
            running_loss = 0
        running_loss += abs(loss.item())
        
        # Every 1 epoch
        
        if args.iter_ckpt:
            if i % args.iter == 0 and i > 0:
                save_path = os.path.join(save_checkpoint_dir, 'netG_{}_{}_{}.pth'.format(save_checkpoint_suffix, epoch, i))
            
                if isinstance(netG, nn.DataParallel):
                    torch.save(netG.module.state_dict(), save_path)
                else:
                    torch.save(netG.state_dict(), save_path)
                
        else:
            if i % 80000 == 0 and i > 0: # 1epoch=80000batch                        
                save_path = os.path.join(save_checkpoint_dir, 'netG_{}_{}.pth'.format(save_checkpoint_suffix, epoch))
    
                if isinstance(netG, nn.DataParallel):
                    torch.save(netG.module.state_dict(), save_path)
                else:
                    torch.save(netG.state_dict(), save_path)